In [1]:
from flask import Flask

import pandas as pd
import requests
import json


from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
def read_api_key(file_name):
    with open(file_name, 'r') as file:
        api_key = file.read().strip()
    return api_key

In [3]:
api_key = read_api_key('../youtube_api_key')

In [4]:
# Credential builder
youtube=build(
        'youtube',
        'v3',
        developerKey=api_key
    )

In [5]:
# Get channel ID by channel name
# Call the search().list() function
request = youtube.search().list(
    part="snippet",
    q="LVPesLOL",  # Replace with your channel ID
    maxResults=1,  # You can adjust this value
    order="date"  # This will sort the videos by date
)
response = request.execute()
channel_id = response['items'][0]['snippet']['channelId']

In [6]:
# Get upload list
request = youtube.channels().list(
    part="contentDetails",
    id=channel_id
)

response = request.execute()

# Get the 'Uploads' playlist ID
uploads_playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

In [7]:
# Get the videos in the 'Uploads' playlist (By default they are in descending cronological order)
request = youtube.playlistItems().list(
    part="snippet",
    playlistId=uploads_playlist_id,
    maxResults=50  # You can adjust this value
)

response = request.execute()

In [8]:
# Get the videos in the 'Uploads' playlist
next_page_token = None
response_list = []
i = 5
while i>0:
    request = youtube.playlistItems().list(
        part="snippet",
        playlistId=uploads_playlist_id,
        maxResults=50,  # You can adjust this value
        pageToken = next_page_token
    )

    response = request.execute()
    response_list.append(response)
    next_page_token = response.get('nextPageToken')
    i = i-1
    if next_page_token is None or i==0:
        break

In [9]:
# Obtain all data needed for the latest videos
df = pd.DataFrame(columns = ['date', 'match_name', 'map_number', 'league', 'split', 'video_id', 'embed_link'])
for response in response_list:
    # compute the embed code for each video
    for item in response['items']:
        title = item['snippet']['title']
        date = item['snippet']['publishedAt'][0:10]
        title_split = title.split(' - ')
        if 'VS' not in title_split[0]:
            continue
        match_name = title_split[0]
        if 'MAPA' in title_split[1]:
            map_number = title_split[1]
        else:
            map_number = 'Mapa 1'

        if 'MAPA' in title_split[1]:
            league = title_split[3]
            split = title_split[4]
        else:
            league = title_split[2]
            split = title_split[3]

        video_id = item['snippet']['resourceId']['videoId']
        embed_link = f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{video_id}" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'
        print(title)
        print(date)
        print(match_name)
        print(map_number)
        print(league)
        print(split)
        print(video_id)
        print(embed_link)
        df = pd.concat([df, pd.DataFrame([[date, match_name, map_number, league, split, video_id, embed_link]],
                                    columns = ['date', 'match_name', 'map_number', 'league', 'split', 'video_id', 'embed_link'])])

LUA GAMING VS HERACLES GAMING - JORNADA 10 - SUPERLIGA SEGUNDA - PRIMAVERA 2024 - LEAGUE OF LEGENDS
2024-03-02
LUA GAMING VS HERACLES GAMING
Mapa 1
SUPERLIGA SEGUNDA
PRIMAVERA 2024
XIxFOh9Tcgs
<iframe width="560" height="315" src="https://www.youtube.com/embed/XIxFOh9Tcgs" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
XOLDIERS VS STORMBRINGERS - JORNADA 10 - SUPERLIGA SEGUNDA - PRIMAVERA 2024 - LEAGUE OF LEGENDS
2024-03-02
XOLDIERS VS STORMBRINGERS
Mapa 1
SUPERLIGA SEGUNDA
PRIMAVERA 2024
XuIcs-qK1oA
<iframe width="560" height="315" src="https://www.youtube.com/embed/XuIcs-qK1oA" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
DANGO VS PRINCIPALITY - JORNADA 10 - SUPERLIGA SEGUNDA - PRIMAVERA 2024 - LEAGUE OF LEGENDS
2024-03-02
DANGO VS PRINCIPALITY
Mapa 1
SUPERLIGA SEGUNDA
PRIMAVERA 2024
wujpUVR0WXs
<ifra

IndexError: list index out of range

In [60]:
df['match_name'] = df['match_name'].str.strip()

In [62]:
df = df.sort_values(by=['date', 'match_name', 'map_number'])

In [64]:
full_match_info_dict = df.to_dict('records')

In [65]:
match_dict = df.reindex(columns=['date', 'match_name']).drop_duplicates().to_dict('records')

In [66]:
date_dict = df.reindex(columns=['date']).drop_duplicates().to_dict('records')

In [67]:
for match_map in full_match_info_dict:
    for match in match_dict:
        if match['date'] == match_map['date'] and match['match_name'] == match_map['match_name']:
            try:
                match['details'].append(match_map)
            except:
                match['details'] = [match_map]

In [68]:
for match in match_dict:
    for date in date_dict:
        if date['date'] == match['date']:
            try:
                date['matches'].append(match)
            except:
                date['matches'] = [match]

In [69]:
i = 20
for date in date_dict:
    print('\n')
    print(date['date'])
    for match in date['matches']:
        print(match['match_name'])
        for match_map in match['details']:
            print(match_map['map_number'])



2024-01-14
FNATIC VS TEAM VITALITY
Mapa 1
G2 ESPORTS VS KARMINE CORP
Mapa 1
GIANTX VS TEAM BDS
Mapa 1
MAD LIONS KOI VS SK GAMING
Mapa 1
ROGUE VS TEAM HERETICS
Mapa 1


2024-01-15
G2 ESPORTS VS ROGUE
Mapa 1
KARMINE CORP VS MAD LIONS KOI
Mapa 1
SK GAMING VS FNATIC
Mapa 1
TEAM HERETICS VS GIANTX
Mapa 1
TEAM VITALITY VS TEAM BDS
Mapa 1


2024-01-17
BARÇA ESPORTS VS CASE ESPORTS
Mapa 1
GIANTX PRIDE VS LOS HERETICS
Mapa 1
REBELS GAMING VS RAMBOOT CLUB
Mapa 1
UCAM ESPORTS VS GUASONES
Mapa 1


2024-01-18
BARÇA ESPORTS VS UCAM ESPORTS
Mapa 1
GUASONES VS MOVISTAR KOI
Mapa 1
LOS HERETICS VS CASE ESPORTS
Mapa 1
MOVISTAR KOI VS ZETA
Mapa 1
REBELS GAMING VS GIANTX PRIDE
Mapa 1
ZETA VS RAMBOOT CLUB
Mapa 1


2024-01-19
ECORP VS LUA GAMING
Mapa 1
HERACLES GAMING VS PRINCIPALITY
Mapa 1
OXIGEN KUMIHO VS XOLDIERS
Mapa 1
STORMBRINGERS VS DANGO
Mapa 1


2024-01-20
G2 ESPORTS VS MAD LIONS KOI
Mapa 1
KARMINE CORP VS GIANTX
Mapa 1
TEAM BDS VS FNATIC
Mapa 1
TEAM HERETICS VS SK GAMING
Mapa 1
TEAM VITALITY VS R

In [35]:
def get_videos_from_youtube_api(api_key):
    # Credential builder
    youtube=build(
            'youtube',
            'v3',
            developerKey=api_key
        )

    # Get channel ID by channel name
    # Call the search().list() function
    request = youtube.search().list(
        part="snippet",
        q="LVPesLOL",  # Replace with your channel ID
        maxResults=1,  # You can adjust this value
        order="date"  # This will sort the videos by date
    )
    response = request.execute()
    channel_id = response['items'][0]['snippet']['channelId']
    
    # Get upload list
    request = youtube.channels().list(
        part="contentDetails",
        id=channel_id
    )
    response = request.execute()

    # Get the 'Uploads' playlist ID
    uploads_playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    # Get the videos in the 'Uploads' playlist (By default they are in descending cronological order)
    request = youtube.playlistItems().list(
        part="snippet",
        playlistId=uploads_playlist_id,
        maxResults=50  # You can adjust this value
    )
    response = request.execute()
    
    # Get the videos in the 'Uploads' playlist
    next_page_token = None
    response_list = []
    i = 5
    while i>0:
        request = youtube.playlistItems().list(
            part="snippet",
            playlistId=uploads_playlist_id,
            maxResults=50,  # You can adjust this value
            pageToken = next_page_token
        )

        response = request.execute()
        response_list.append(response)
        next_page_token = response.get('nextPageToken')
        i = i-1
        if next_page_token is None or i==0:
            break
    # Obtain all data needed for the latest videos
    df = pd.DataFrame(columns = ['date', 'match_name', 'map_number', 'league', 'split', 'video_id', 'embed_link', 'datetime'])
    for response in response_list:
        # compute the embed code for each video
        for item in response['items']:
            title = item['snippet']['title']
            date = item['snippet']['publishedAt'][0:10]
            datetime = item['snippet']['publishedAt']
            title_split = title.split(' - ')
            try:
                if 'VS' not in title_split[0]:
                    continue
                match_name = title_split[0]
                if 'MAPA' in title_split[1]:
                    map_number = title_split[1]
                else:
                    map_number = 'Mapa 1'

                if 'MAPA' in title_split[1]:
                    league = title_split[3]
                    split = title_split[4]
                else:
                    league = title_split[2]
                    split = title_split[3]

                video_id = item['snippet']['resourceId']['videoId']
                embed_link = f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{video_id}" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>'

                df = pd.concat([df, pd.DataFrame([[date, match_name, map_number, league, split, video_id, embed_link, datetime]],
                                            columns = ['date', 'match_name', 'map_number', 'league', 'split', 'video_id', 'embed_link', 'datetime'])])
            except:
                print(title_split)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.reset_index().iloc[:,1:]
    return df

In [36]:
df = get_videos_from_youtube_api(api_key)

['MAD LIONS KOI VS TEAM HERETICS | LEC 2024 | WINTER SPLIT | LEAGUE OF LEGENDS | SEMANA 1', 'DÍA 1 | #']
['VITALITY VS GIANTX | LEC 2024 | WINTER SPLIT | LEAGUE OF LEGENDS | SEMANA 1', 'DÍA 1 | #LECINVIERNO']
['FNATIC VS KARMINE CORP | LEC 2024 | WINTER SPLIT | LEAGUE OF LEGENDS | SEMANA 1', 'DÍA 1 | #LECINVI']
['ROGUE VS SK GAMING | LEC 2024 | WINTER SPLIT | LEAGUE OF LEGENDS | SEMANA 1', 'DÍA 1 | #LECINVIERNO']
['G2 ESPORTS VS TEAM BDS | LEC 2024 | WINTER SPLIT | LEAGUE OF LEGENDS | SEMANA 1', 'DÍA 1 | #LECINVIE']


In [33]:
df.sort_values(by=['datetime', 'match_name', 'map_number']).to_json()

ValueError: DataFrame index must be unique for orient='columns'.

In [50]:
from json import loads, dumps
result = df.to_json(orient="columns")
parsed = loads(result)
parsed

{'date': {'0': '2024-02-29',
  '1': '2024-02-29',
  '2': '2024-02-28',
  '3': '2024-02-28',
  '4': '2024-02-28',
  '5': '2024-02-28',
  '6': '2024-02-28',
  '7': '2024-02-27',
  '8': '2024-02-27',
  '9': '2024-02-27',
  '10': '2024-02-27',
  '11': '2024-02-23',
  '12': '2024-02-23',
  '13': '2024-02-23',
  '14': '2024-02-23',
  '15': '2024-02-22',
  '16': '2024-02-22',
  '17': '2024-02-22',
  '18': '2024-02-22',
  '19': '2024-02-22',
  '20': '2024-02-21',
  '21': '2024-02-21',
  '22': '2024-02-21',
  '23': '2024-02-21',
  '24': '2024-02-21',
  '25': '2024-02-20',
  '26': '2024-02-20',
  '27': '2024-02-20',
  '28': '2024-02-20',
  '29': '2024-02-20',
  '30': '2024-02-19',
  '31': '2024-02-19',
  '32': '2024-02-19',
  '33': '2024-02-19',
  '34': '2024-02-18',
  '35': '2024-02-18',
  '36': '2024-02-18',
  '37': '2024-02-18',
  '38': '2024-02-17',
  '39': '2024-02-17',
  '40': '2024-02-17',
  '41': '2024-02-17',
  '42': '2024-02-16',
  '43': '2024-02-16',
  '44': '2024-02-16',
  '45': '202